# Data Cleaning

## Renaming Column Name

In [127]:
import pandas as pd

sogie = pd.read_csv('SOGIE BILL.csv')

sogie

,Timestamp,Respondents' Consent,What is your name? (optional),Which college department are you part of?,What is your current college year?,Are you already aware of what SOGIE Equality Bill is?,How much knowledge would you describe yourself having regarding LGBTQIA+?,What is your assigned sex at birth?,What is your Gender Identity?,How would you describe your Gender Expression?,...,Will SOGIE Bill help promote programs for non-discrimination and diversity in schools and workplace?,Will SOGIE Bill penalize employers who discriminate against LGBTQIA+ members?,Will SOGIE Bill provide programs to protect students in school?,Will SOGIE Bill help LGBTQIA+ members to know their rights?,Will SOGIE Bill help the LGBTQIA+ members to gain more self-confidence?,Will SOGIE Bill help the implementation of LGBTQIA+ members own public comfort room?,Will SOGIE Bill help LGBTQIA+ members to be more open about their gender identity?,Will SOGIE Bill lessen the bullying cases between LGBTQIA+ student and an ordinary student?,Do you think the SOGIE Bill is for the legalization of Same-Sex Marriage?,Do you think that SOGIE Bill is only for the LGBTQIA+ community?
0,2024/02/19 11:00:31 AM GMT+8,I agree to participate to this survey,CSCS30001,CSCS,3rd Year,Yes,I know some basic knowledge about LGBTQIA+,Female,Woman,Feminine,...,4,3,3,5,5,4,5,4,5,3
1,2024/02/19 11:04:26 AM GMT+8,I agree to participate to this survey,CSCS30002,CSCS,3rd Year,Yes,I am fully knowledgeable about LGBTQIA+,Male,Man,Masculine,...,4,2,3,5,5,3,3,3,4,1
2,2024/02/19 11:06:45 AM GMT+8,I agree to participate to this survey,CSCS30003,CSCS,3rd Year,No,I know nothing at all,Male,Man,Masculine,...,1,1,1,1,1,1,1,1,1,1
3,2024/02/19 11:10:21 AM GMT+8,I agree to participate to this survey,CSCS30004,CSCS,3rd Year,Yes,I know some basic knowledge about LGBTQIA+,Female,Woman,Feminine,...,4,4,4,4,4,4,4,3,2,1
4,2024/02/19 11:12:19 AM GMT+8,I agree to participate to this survey,CSCS30005,CSCS,3rd Year,No,I know some basic knowledge about LGBTQIA+,Female,Woman,Feminine,...,5,3,3,5,3,2,4,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,2024/02/20 5:02:46 PM GMT+8,I agree to participate to this survey,CTHM30037,CTHM,3rd Year,Yes,I know some basic knowledge about LGBTQIA+,Female,Woman,Androgynous/Unisexual,...,5,2,3,5,5,5,3,4,1,1
314,2024/02/20 5:04:33 PM GMT+8,I agree to participate to this survey,CTHM30038,CTHM,3rd Year,Yes,I know nothing at all,Female,Woman,Prefer not to disclose,...,3,3,3,4,3,3,3,2,3,3
315,2024/02/20 5:05:42 PM GMT+8,I agree to participate to this survey,CTHM30039,CTHM,3rd Year,Yes,I know some basic knowledge about LGBTQIA+,Female,Woman,Feminine,...,5,4,5,5,5,4,5,3,4,4
316,2024/02/20 5:06:52 PM GMT+8,I agree to participate to this survey,CTHM30040,CTHM,3rd Year,Yes,I am fully knowledgeable about LGBTQIA+,Male,Man,Androgynous/Unisexual,...,3,2,4,4,3,4,4,3,4,4


In [128]:

# REMOVE COLUMNS 'RESPONDENTS CONSET' AND 'TIMESTAMP'
sogie_cleaned = sogie.drop(["Respondents' Consent", 'Timestamp'], axis=1)

# ADD NUMBER COLUMN
indx_col_values = [x + 1 for x in range(len(sogie_cleaned))]
sogie_cleaned.insert(loc=0, column='NUM', value=indx_col_values)

# RENAME COLUMNS
sogie_cleaned.rename(columns = 
                    {
                        "Respondents' Consent":'CONSENT',
                        "What is your name? (optional)": 'RESPONDENT_ID',
                        "Which college department are you part of?": 'COLLEGE',
                        "What is your current college year?": 'YEAR_LEVEL',
                        "Are you already aware of what SOGIE Equality Bill is?": 'SOGIE_VS_NO_SOGIE',
                        "How much knowledge would you describe yourself having regarding LGBTQIA+?": 'NONE_BASIC_FULL',
                        "What is your assigned sex at birth?": 'SEX',
                        "What is your Gender Identity?": 'GENDER_IDENTITY',
                        "How would you describe your Gender Expression?": 'GENDER_EXPRESSION',
                        "How would you describe your Sexual Orientation": 'SEXUAL_EXPRESSION'
                    }, inplace = True)

respondents = sogie_cleaned.iloc[:,:10]
questions = pd.DataFrame({
    'QUESTION_ID': [x for x in range(len(sogie_cleaned.columns[10:]))],
    'QUESTION': [x for x in sogie_cleaned.columns[10:]]
})

## Separating by Tables

In [129]:
mask_sex = [
    respondents['SEX'] == 'Prefer not to disclose'
]

mask_Orientation = [
    respondents['SEXUAL_EXPRESSION'] == 'I am ONLY attracted to men',
    respondents['SEXUAL_EXPRESSION'] == 'I am ONLY attracted to women',
    respondents['SEXUAL_EXPRESSION'] == 'I am NOT sexually attracted to anyone',
    respondents['SEXUAL_EXPRESSION'] == 'I am attracted to anyone REGARDLESS of their gender',
    respondents['SEXUAL_EXPRESSION'] == 'Prefer not to disclose'
]

mask_Expression = [
    respondents['GENDER_EXPRESSION'] == 'Androgynous/Unisexual',
    respondents['GENDER_EXPRESSION'] == 'Prefer not to disclose'
]

mask_Identity   = [
    respondents['GENDER_IDENTITY'] == 'Gender Neutral (neither)',
    respondents['GENDER_IDENTITY'] == 'Gender Queer (both)',
    respondents['GENDER_IDENTITY'] == 'Prefer not to disclose'
]

mask_Awareness   = [
    respondents['NONE_BASIC_FULL'] == 'I know some basic knowledge about LGBTQIA+',
    respondents['NONE_BASIC_FULL'] == 'I am fully knowledgeable about LGBTQIA+',
    respondents['NONE_BASIC_FULL'] == 'I know nothing at all'
]

mask_Year  = [
    respondents['YEAR_LEVEL'] == '1st Year',
    respondents['YEAR_LEVEL'] == '2nd Year',
    respondents['YEAR_LEVEL'] == '3rd Year',
    respondents['YEAR_LEVEL'] == '4th Year'
]

# NORMALIZE SEX
respondents.loc[mask_sex[0], 'SEX'] = None

# NORMALIZE SEXUAL EXPRESSION
respondents.loc[mask_Orientation[0], 'SEXUAL_EXPRESSION'] = 'Men'
respondents.loc[mask_Orientation[1], 'SEXUAL_EXPRESSION'] = 'Women'
respondents.loc[mask_Orientation[2], 'SEXUAL_EXPRESSION'] = 'Asexual'
respondents.loc[mask_Orientation[3], 'SEXUAL_EXPRESSION'] = 'Pansexual'
respondents.loc[mask_Orientation[4], 'SEXUAL_EXPRESSION'] = None

# NORMALIZE GENDER EXPRESSION
respondents.loc[mask_Expression[0], 'GENDER_EXPRESSION'] = 'Androgynous'
respondents.loc[mask_Expression[1], 'GENDER_EXPRESSION'] = None

# NORMALIZE GENDER IDENTITY
respondents.loc[mask_Identity[0], 'GENDER_IDENTITY'] = 'Neutral'
respondents.loc[mask_Identity[1], 'GENDER_IDENTITY'] = 'Queer'
respondents.loc[mask_Identity[2], 'GENDER_IDENTITY'] = None

# NORMALIZE AWARENESS
respondents.loc[mask_Awareness[0], 'NONE_BASIC_FULL'] = 'Basic'
respondents.loc[mask_Awareness[1], 'NONE_BASIC_FULL'] = 'Full'
respondents.loc[mask_Awareness[2], 'NONE_BASIC_FULL'] = 'None'

# NORMALIZE SEXUAL EXPRESSION
respondents.loc[mask_Year[0], 'YEAR_LEVEL'] = '1'
respondents.loc[mask_Year[1], 'YEAR_LEVEL'] = '2'
respondents.loc[mask_Year[2], 'YEAR_LEVEL'] = '3'
respondents.loc[mask_Year[3], 'YEAR_LEVEL'] = '4'
respondents

,NUM,RESPONDENT_ID,COLLEGE,YEAR_LEVEL,SOGIE_VS_NO_SOGIE,NONE_BASIC_FULL,SEX,GENDER_IDENTITY,GENDER_EXPRESSION,SEXUAL_EXPRESSION
0,1,CSCS30001,CSCS,3,Yes,Basic,Female,Woman,Feminine,Men
1,2,CSCS30002,CSCS,3,Yes,Full,Male,Man,Masculine,Men
2,3,CSCS30003,CSCS,3,No,None,Male,Man,Masculine,Women
3,4,CSCS30004,CSCS,3,Yes,Basic,Female,Woman,Feminine,Men
4,5,CSCS30005,CSCS,3,No,Basic,Female,Woman,Feminine,Asexual
...,...,...,...,...,...,...,...,...,...,...
313,314,CTHM30037,CTHM,3,Yes,Basic,Female,Woman,Androgynous,Men
314,315,CTHM30038,CTHM,3,Yes,None,Female,Woman,None,Pansexual
315,316,CTHM30039,CTHM,3,Yes,Basic,Female,Woman,Feminine,Men
316,317,CTHM30040,CTHM,3,Yes,Full,Male,Man,Androgynous,Women


In [130]:
questions

,QUESTION_ID,QUESTION
0,0,Will SOGIE Bill prevent the discrimination bas...
1,1,Will SOGIE Bill give equal access to LGBTQIA+ ...
2,2,Will SOGIE Bill help promote programs for non-...
3,3,Will SOGIE Bill penalize employers who discrim...
4,4,Will SOGIE Bill provide programs to protect st...
5,5,Will SOGIE Bill help LGBTQIA+ members to know ...
6,6,Will SOGIE Bill help the LGBTQIA+ members to g...
7,7,Will SOGIE Bill help the implementation of LGB...
8,8,Will SOGIE Bill help LGBTQIA+ members to be mo...
9,9,Will SOGIE Bill lessen the bullying cases betw...


In [131]:
questions_list = questions['QUESTION'].values

In [132]:
answers_dict = {
    'RESPONDENT_ID': [],
    'QUESTION_ID': [],
    'ANSWER': []
}
for index, row in sogie_cleaned.iterrows():
    for indx, quest in questions.iterrows():
        answers_dict['RESPONDENT_ID'].append(row['RESPONDENT_ID'])
        answers_dict['QUESTION_ID'].append(quest['QUESTION_ID'])
        answers_dict['ANSWER'].append(row[quest['QUESTION']])

answers = pd.DataFrame(answers_dict)
answers

,RESPONDENT_ID,QUESTION_ID,ANSWER
0,CSCS30001,0,4
1,CSCS30001,1,4
2,CSCS30001,2,4
3,CSCS30001,3,3
4,CSCS30001,4,3
...,...,...,...
3811,CTHM30041,7,1
3812,CTHM30041,8,1
3813,CTHM30041,9,1
3814,CTHM30041,10,1


# DATA VALIDATION

## Check for duplicates

In [133]:
from collections import defaultdict

resp_ids = respondents['RESPONDENT_ID'].values
counts = defaultdict(int)

# Count occurrences of each element in the list
for item in resp_ids:
    counts[item] += 1

# Print elements with counts greater than 1 (duplicates)
for item, count in counts.items():
    if count > 1:
        print(f"{item} is duplicated {count} times")

CSCS20041 is duplicated 2 times


## Check for Inconsistencies

# Data Exportation

In [144]:
college_df = respondents[['COLLEGE']]
num_CSCS = len(college_df[college_df['COLLEGE'] == 'CSCS'])
num_CTHM = len(college_df[college_df['COLLEGE'] == 'CTHM'])
num_CLAC = len(college_df[college_df['COLLEGE'] == 'CLAC'])
ratio_college_df = pd.DataFrame({
    'Department': ['CSCS', 'CTHM', 'CLAC'],
    'No. Respondents': [num_CSCS, num_CTHM, num_CLAC]
})
ratio_college_df

ratio_college_df.to_csv('./intro/COLLEGE.csv', index=False)

In [158]:
sex_list = list(set(respondents['SEX'].values))
sex_list.remove(None)

ratio_sex_df = pd.DataFrame({
    'Sex at Birth': sex_list,
    'No. Respondents': [len(respondents[respondents['SEX'] == sex]) for sex in sex_list]
})

ratio_sex_df.to_csv('./intro/SEX.csv', index=False)

In [159]:
gender_list = list(set(respondents['GENDER_IDENTITY'].values))
gender_list.remove(None)

ratio_gender_df = pd.DataFrame({
    'Gender Identity': gender_list,
    'No. Respondents': [len(respondents[respondents['GENDER_IDENTITY'] == gen]) for gen in gender_list]
})

ratio_gender_df.to_csv('./intro/GENDER.csv', index=False)

In [134]:
answers.to_csv('SOGIE_Answers.csv', index=False)
questions.to_csv('SOGIE_Questions.csv', index=False)
respondents.to_csv('SOGIE_Respondents.csv', index=False)